# CI/CD for Titanic Survival Prediction API

This notebook demonstrates how to set up Continuous Integration and Continuous Deployment (CI/CD) for the Dockerized Titanic survival prediction FastAPI app using GitHub Actions.

## CI/CD Workflow Overview

- **Continuous Integration (CI):** Automatically build, test, and lint the app on every push or pull request.
- **Continuous Deployment (CD):** Build and push the Docker image to a container registry (e.g., Docker Hub or GitHub Container Registry) and optionally deploy to a cloud service.

## Setting Up GitHub Actions

GitHub Actions uses workflow files stored in the `.github/workflows/` directory of your repository. These files define the steps for CI/CD.

In [ ]:
# Write a sample GitHub Actions workflow for CI/CD
workflow = '''
name: CI/CD for Titanic API

on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

jobs:
  build-test:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - name: Set up Python
        uses: actions/setup-python@v4
        with:
          python-version: '3.10'
      - name: Install dependencies
        run: |
          python -m pip install --upgrade pip
          pip install -r s23-model-deployment/requirements.txt
      - name: Lint with flake8
        run: |
          pip install flake8
          flake8 s23-model-deployment/titanic_api.py
      - name: Test Docker build
        run: |
          docker build -t titanic-api s23-model-deployment/

  docker-push:
    needs: build-test
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - name: Log in to Docker Hub
        uses: docker/login-action@v3
        with:
          username: ${{ secrets.DOCKERHUB_USERNAME }}
          password: ${{ secrets.DOCKERHUB_TOKEN }}
      - name: Build and push Docker image
        run: |
          docker build -t ${{ secrets.DOCKERHUB_USERNAME }}/titanic-api:latest s23-model-deployment/
          docker push ${{ secrets.DOCKERHUB_USERNAME }}/titanic-api:latest
'''

import os
os.makedirs('.github/workflows', exist_ok=True)
with open('.github/workflows/ci-cd.yml', 'w') as f:
    f.write(workflow)
print('GitHub Actions workflow file created at .github/workflows/ci-cd.yml')

## Notes on Secrets and Triggering

- Store your Docker Hub credentials as repository secrets: `DOCKERHUB_USERNAME` and `DOCKERHUB_TOKEN`.
- The workflow runs on every push or pull request to the `main` branch.
- You can extend the workflow to deploy to cloud services (e.g., AWS, Azure, GCP) after pushing the Docker image.